# Музыка больших городов

Сравнение Москвы и Петербурга окружено мифами. Например:
 * Москва — мегаполис, подчинённый жёсткому ритму рабочей недели;
 * Петербург — культурная столица, со своими вкусами.

Сравним поведение пользователей двух столиц на данных Яндекс Музыки.

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**План исследования**

Данные о поведении пользователей находятся в файле `yandex_music_project.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных.

Нужно проверить данные на ошибки и оценить их влияние на результаты исследования. Затем на этапе предобработки вы по возможности исправить наиболее критичные ошибки данных.

Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

Составим первое представление о данных Яндекс Музыки.

In [1]:
# импорт библиотеки pandas
import pandas as pd

In [2]:
# чтение файла с данными и сохранение в df
df = pd.read_csv('https://<LINK_REMOVED>/datasets/yandex_music_project.csv')

In [3]:
# получение первых 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [4]:
# получение общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В таблице семь столбцов. Тип данных во всех столбцах — `object`.
Согласно документации к данным:
- `userID` — идентификатор пользователя;
- `Track` — название трека;
- `artist` — имя исполнителя;
- `genre` — название жанра;
- `City` — город пользователя;
- `time` — время начала прослушивания;
- `Day` — день недели.

В названиях столбцов `userID` и `City` есть лишние пробелы. Кроме того, названия столбцов - `userID`, `Track`, `City` и `Day` не соответствуют хорошему стилю, т.к. сочетают заглавные и строчные буквы. Кроме того, слова в названиях столбцов должны быть разделены знаком подчёркивания.

Количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Переименование столбцов

In [5]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
# переименование столбцов, названия которых не соответствуют хорошему стилю
df = df.rename(columns={
    '  userID': 'user_id',
    'Track': 'track',
    '  City  ': 'city',
    'Day': 'day'
})

In [7]:
# проверка результатов - перечень названий столбцов
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Обработка пропущенных значений

In [8]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропущенные значения есть в столбцах `track`, `artist` и `genre`.

Не все пропущенные значения влияют на исследование. Так, в пропуски в столбцах `track` и `artist` не повлияют на результаты исследования, поэтому достаточно заменить их явными обозначениями.

Однако пропуски в столбце `genre` могут помешать сравнить музыкальные вкусы москвичей и петербуржцев. На практике было бы правильно установить причину пропусков и восстановить данные. Такой возможности нет в учебном проекте. Поэтому придётся:
- заполнить и эти пропуски явными обозначениями;
- оценить, насколько они повредят расчётам.

In [9]:
# подсчёт явных дубликатов
print('Количество дубликатов:', df.duplicated().sum())

Количество дубликатов: 3826


В данных присутствуют дубликаты - повторяющиеся строки. Их следует удалить, полезной информации они не несут.

In [10]:
# удаление полных дубликатов
df = df.drop_duplicates(ignore_index=True)

# проверка на отсутствие полных дубликатов
df.duplicated().sum()

0

Полные дубликаты устранены.

In [11]:
# подсчёт пропусков после удаления полных дубликатов
df.isna().sum()

user_id       0
track      1155
artist     6772
genre      1127
city          0
time          0
day           0
dtype: int64

Вывод: Количество пропусков после удаления полных дубликатов изменилось, значит в исходных данных были дубликаты строк с пропусками. Однако по-прежнему остаются пропуски в столбцах `track`, `artist` и `genre`.

In [12]:
# распределение пропусков в столбце genre по городам
df[df['genre'].isna()].groupby('city')['user_id'].count()

city
Moscow              849
Saint-Petersburg    278
Name: user_id, dtype: int64

In [13]:
# распределение пропусков в столбце genre по дням недели
df[df['genre'].isna()].groupby('day')['user_id'].count()

day
Friday       358
Monday       488
Wednesday    281
Name: user_id, dtype: int64

In [14]:
# распределение пропусков в столбце genre по городам и дням недели
df_genre_isna = (
    df[df['genre'].isna()]
    .groupby(['city','day'])['user_id']
    .count()
    .unstack('day')
    .iloc[:, [1, 2, 0]]
)
df_genre_isna

day,Monday,Wednesday,Friday
city,,,
Moscow,393,191,265
Saint-Petersburg,95,90,93


In [15]:
# замена пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [16]:
# проверка на отсутствие пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Теперь ни в одном столбце таблицы нет пропущенных значений.

### Обработка дубликатов

In [17]:
# подсчёт полных дубликатов
df.duplicated().sum()

0

После заполнения пропусков новые дубликаты в данных не появились.

In [18]:
# просмотр уникальных отсортированных названий жанров
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Название одного и того же жанра может быть записано немного по-разному. Такие ошибки тоже повлияют на результат исследования.

Обнаружены неявные дубликаты жанров `hiphop`, `electronic` и `pop`.

In [19]:
# устранение неявных дубликатов жанра hiphop
df['genre'] = df['genre'].replace(['hip', 'hop', 'hip-hop'], 'hiphop')

In [20]:
# устранение неявных дубликатов жанра electronic
# 'электроника' = 'electronic' по-русски
df['genre'] = df['genre'].replace('электроника', 'electronic')

In [21]:
# устранение неявных дубликатов жанра pop
# 'ïîï' в кодировке Windows-1252 = 'поп' в кодировке Windows-1251 = 'pop' по-русски
df['genre'] = df['genre'].replace('ïîï', 'pop')

In [22]:
# проверка на отсутствие неявных дубликатов
df['genre'].sort_values().unique()

array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Неявные дубликаты жанров *hiphop*, *electronic* и *pop* устранены.

**Выводы**

Предобработка обнаружила три проблемы в данных:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

Заголовки столбцов исправлены заголовки, чтобы упростить работу с таблицей. 

Полные дубликаты строк удалены. Также устранены выявленные неявные дубликаты названий жанров. Без дубликатов исследование станет более точным.

Пропущенные значения заменены на `'unknown'`. Ещё предстоит увидеть, не повредят ли исследованию пропуски в колонке `genre`.

Теперь можно перейти к проверке гипотез.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. Для этого:

* Разделим пользователей Москвы и Санкт-Петербурга.
* Сравним, сколько треков послушала каждая группа пользователей в понедельник, среду и пятницу.


In [23]:
# подсчёт прослушиваний в каждом городе
df.groupby('city')['user_id'].count()

city
Moscow              42741
Saint-Petersburg    18512
Name: user_id, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Однако из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

In [24]:
# подсчёт прослушиваний в каждый из трёх дней
df.groupby('day')['user_id'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: user_id, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

In [25]:
# подсчёт прослушиваний в каждом городе в каждый из дней
(df.groupby(['city', 'day'])['user_id'].count()
   # переводим 2-й уровень индекса в столбец и меняем порядок столбцов
   .unstack('day').iloc[:, [1, 2, 0]])

day,Monday,Wednesday,Friday
city,,,
Moscow,15740,11056,15945
Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

In [26]:
# получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'
moscow_general = df[df['city'] == 'Moscow']

In [27]:
# получение таблицы spb_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Saint-Petersburg'
spb_general = df[df['city'] == 'Saint-Petersburg']

In [28]:
# времена суток 
times = {
    'morning': ['07:00:00', '11:00:00'], 
    'evening': ['17:00:00', '23:00:00'],
    'whole day': ['00:00:00', '23:59:59']
}
# количество позиций в рейтинге
top_n = 10

In [29]:
def genre_weekday(df, day, time1, time2):
    result = (
        df
        .query("day == @day and @time1 < time < @time2")
        .groupby('genre')['user_id'].count()
        .sort_values(ascending=False)[:top_n]
        .reset_index()
        .rename(columns={'user_id': 'play_count'})
    )

    return result

In [30]:
# функция, которая возвращает датафрейм со сравнением рейтингов по городам в заданный день и время суток
def genre_weekday_table(df, cities, day, daytime='whole day', in_percent=False):
    data = []
    for city in cities:
        df_city = df[df['city'] == city]
        time1, time2 = times.get(daytime, times['whole day'])
        item = genre_weekday(df_city, day, time1, time2)
        if in_percent:
            total_tracks = item['play_count'].sum()
            item['play_count'] /= total_tracks
            item['play_count'] = item['play_count'].mul(100).round(2)
            item = item.rename(columns={'play_count': 'percent'})
        data.append(item)
    result = pd.concat(data, axis='columns', keys=cities)
    return result

In [31]:
cities = ['Moscow', 'Saint-Petersburg']

In [32]:
# сравнение топ-10 жанров в понедельник утром
genre_weekday_table(df, cities, 'Monday', 'morning')

Moscow            Saint-Petersburg           
         genre play_count            genre play_count
0          pop        781              pop        218
1        dance        549            dance        182
2   electronic        480             rock        162
3         rock        474       electronic        147
4       hiphop        286           hiphop         80
5       ruspop        186           ruspop         64
6        world        181      alternative         58
7       rusrap        175           rusrap         55
8  alternative        164             jazz         44
9      unknown        161        classical         40

In [33]:
# сравнение топ-10 жанров в пятницу вечером
genre_weekday_table(df, cities, 'Friday', 'evening')

Moscow            Saint-Petersburg           
         genre play_count            genre play_count
0          pop        713              pop        256
1         rock        517       electronic        216
2        dance        495             rock        216
3   electronic        482            dance        210
4       hiphop        273           hiphop         97
5        world        208      alternative         63
6       ruspop        170             jazz         61
7  alternative        163        classical         60
8    classical        163           rusrap         59
9       rusrap        142            world         54

Сравнение двух городов будет информативнее, если количество прослушиваний в таблицах представить в относительных единицах - процентах от общего количества прослушиваний в заданный временной промежуток в данном городе.

In [34]:
# сравнение топ-10 жанров в понедельник утром в процентах
genre_weekday_table(df, cities, 'Monday', 'morning', in_percent=True)

Moscow         Saint-Petersburg        
         genre percent            genre percent
0          pop   22.72              pop   20.76
1        dance   15.97            dance   17.33
2   electronic   13.97             rock   15.43
3         rock   13.79       electronic   14.00
4       hiphop    8.32           hiphop    7.62
5       ruspop    5.41           ruspop    6.10
6        world    5.27      alternative    5.52
7       rusrap    5.09           rusrap    5.24
8  alternative    4.77             jazz    4.19
9      unknown    4.68        classical    3.81

In [35]:
# сравнение топ-10 жанров в пятницу вечером
genre_weekday_table(df, cities, 'Friday', 'evening', in_percent=True)

Moscow         Saint-Petersburg        
         genre percent            genre percent
0          pop   21.44              pop   19.81
1         rock   15.54       electronic   16.72
2        dance   14.88             rock   16.72
3   electronic   14.49            dance   16.25
4       hiphop    8.21           hiphop    7.51
5        world    6.25      alternative    4.88
6       ruspop    5.11             jazz    4.72
7  alternative    4.90        classical    4.64
8    classical    4.90           rusrap    4.57
9       rusrap    4.27            world    4.18

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное различие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым:

1. И в понедельник утром, и в пятницу вечером топ-10 жанров в Москве и Петербурге практически полностью совпадают. 

2. В Петербурге в топ-10 в обоих случаях попал джаз, а в Москве - нет.

3. В Москве жанр "world" популярнее, чем в Петербурге.


Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [36]:
# функция возвращает n самых популярных жанров для заданного города и количество прослушиваний для каждого жанра
def top_genres(df, city, n=10, in_percent=False):
    df_city = df[df['city'] == city]
    tracks_by_genre = (
        df_city
        .groupby('genre')['user_id'].count()
        .sort_values(ascending=False)
        .head(n)
        .rename('play_count')
    )
    if in_percent:
        total_tracks = df_city['genre'].count()
        # tracks_by_genre /= total_tracks/100
        # tracks_by_genre = tracks_by_genre.rename('percent')

        tracks_by_genre /= total_tracks
        tracks_by_genre = tracks_by_genre.mul(100).round(2)
        tracks_by_genre = tracks_by_genre.rename('percent')
        
    return tracks_by_genre

In [37]:
# сравнение топ-10 жанров в Москве и Петербурге
top_genres_moscow = top_genres(df, 'Moscow').reset_index()
top_genres_spb = top_genres(df, 'Saint-Petersburg').reset_index()
pd.concat([top_genres_moscow, top_genres_spb], axis='columns', keys=cities)

Moscow            Saint-Petersburg           
         genre play_count            genre play_count
0          pop       5893              pop       2431
1        dance       4435            dance       1932
2         rock       3965             rock       1879
3   electronic       3786       electronic       1737
4       hiphop       2096           hiphop        960
5    classical       1616      alternative        649
6        world       1432        classical        646
7  alternative       1379           rusrap        564
8       ruspop       1372           ruspop        538
9       rusrap       1161            world        515

Сравнение двух городов будет информативнее, если количество прослушиваний в таблицах представить в относительных единицах - процентах от общего количества прослушиваний в заданный временной промежуток в данном городе.

In [38]:
# сравнение топ-10 жанров в Москве и Петербурге в процентах от общего количества прослушиваний по каждому городу
top_genres_moscow = top_genres(df, 'Moscow', in_percent=True).reset_index()
top_genres_spb = top_genres(df, 'Saint-Petersburg', in_percent=True).reset_index()
pd.concat([top_genres_moscow, top_genres_spb], axis='columns', keys=cities)

Moscow         Saint-Petersburg        
         genre percent            genre percent
0          pop   13.79              pop   13.13
1        dance   10.38            dance   10.44
2         rock    9.28             rock   10.15
3   electronic    8.86       electronic    9.38
4       hiphop    4.90           hiphop    5.19
5    classical    3.78      alternative    3.51
6        world    3.35        classical    3.49
7  alternative    3.23           rusrap    3.05
8       ruspop    3.21           ruspop    2.91
9       rusrap    2.72            world    2.78

**Выводы**
1. Разница в популярности разных жанров между городами составляет менее 1 %. Поэтому проверка гипотезы, что Москва и Петербург предпочитают разные жанры музыки, требует дополнительных оценок статистической значимости полученных различий.

2. Тем не менее, из полученных результатов действительно следует, что в Москве немного чаще слушают музыку в жанре "поп" и "русский поп", чем в Петербурге, а жанры "русский рэп" и "хипхоп" в Петербурге немного популярнее, чем в Москве.

3. Таким образом, гипотеза частично подтвердилась:
    * Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
    * Вопреки ожиданиям, рэп почти одинаково популярен в Москве и Петербурге.

## Итоги исследования

В ходе исследования проверены три гипотезы. В результате установлено:

1. День недели по-разному влияет на активность пользователей в Москве и Петербурге.

   Первая гипотеза полностью подтвердилась.

2. Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург. Небольшие различия заметны в начале недели, по понедельникам:
    * в Москве слушают музыку жанра “world”,
    * в Петербурге — джаз и классику.

   Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. Во вкусах пользователей Москвы и Петербурга больше общего, чем различий. Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

   Третья гипотеза не подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.

**На практике исследования содержат проверки статистических гипотез.**
Из части данных одного сервиса невозможно сделать какие-то выводы о всех пользователях сервиса без методов статистики.
Проверки статистических гипотез покажут, насколько они достоверны, исходя из имеющихся данных.